In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Загрузка и препроцессинг данных

Нейросети не работают с сырыми даными по типу JPEG или CSV. Вместо этого они оперируют **векторами** и **специальными** представлениями:
* Текстовые файлы должны быть прочитаны, строки получены и разбиты на слова. Слова должны быть проиндексированы и превращены в целочисленные тензоры.
* Изображения должны быть так же прочитаны и преобразованы в тензоры пикселей (целочисленные), а затем нормализованы до некоторых маленьких значений (часто 0-1).
* Табличные данные должны быть обработаны: категоризованные признаки должны быть приведены в целочисленные тензоры, а признаки в виде чисел с плавающей точкой преобразованы в соответствующие тензоры с плавающей точкой. В конце все признаки точно так же должны быть нормализованы, т.к. этого требуют модели.
* Ну и т.д.

## Загрузка данных

Собственно, Keras оперирует тремя типами "входов" (или входных данных):
* NumPy массивами (те самые `ndarray`), точно так же, как это делает sklearn. Это хороший вариант, если все данные умещаются в RAM.
* Tensorflow Dataset объекты. Это вариант подходит для тех случаев, когда данные не помещаются в память и будут считывать либо с диска, либо вообще с распределенной ФС.
* Питоновские генераторы, которые отдают пакеты данных (очевидно,  всё, что реализовано как генератор, может передаваться в Keras).

Важно отметить, что если датасет правда большой, а вы собираетесь обучать модель на GPU, то стоит использовать объект Tensorflow Dataset, т.к. он обеспечивает высокую производительность засчёт:
* ассинхронной предобработки данных на CPU, пока GPU загружена, и постановка предобработанных данных в очередь
* поставки данных в GPU память так, что она немедленно становится доступна для GPU в момент, когда он прекратил обработку предыдущего поднабора данных. Это позволяет использовать GPU на полную.

Keras позволяет использовать ряд утилит для загрузки сырых данных с диска и превращения их сразу в объект `Dataset`:
* `tf.keras.preprocessing.image_dataset_from_directory` - для превращения изображений, отсортированных по специальной классовой структуре (в виде структуры папок) в размеченный датасет тензоров изображений.
* `tf.keras.preprocessing.text_dataset_from_directory` - то же самое, но для текстов

Как пример просто загрузим [вот этот датасет](https://www.kaggle.com/olgabelitskaya/tomato-cultivars) и распределим его по директориям (каждая директория представляет собой отдельный конкретный класс). Теперь можем воспользоваться методом выше, чтобы автоматически создать размеченный датасет из структуры папок.

Моя структура папок:
```
Tomato Cultivars/
...tomato_1/
......01_001.png
......01_002.png
................
...tomato_2/
......02_001.png
......02_002.png
................
............
...tomato_15/
......15_001.png
......15_002.png
................
```

In [8]:
main_directory = '/home/lpshkn/Datasets/Tomato Cultivars/'

In [9]:
dataset = keras.preprocessing.image_dataset_from_directory(main_directory)

Found 776 files belonging to 15 classes.


In [10]:
dataset.class_names

['tomato_1',
 'tomato_10',
 'tomato_11',
 'tomato_12',
 'tomato_13',
 'tomato_14',
 'tomato_15',
 'tomato_2',
 'tomato_3',
 'tomato_4',
 'tomato_5',
 'tomato_6',
 'tomato_7',
 'tomato_8',
 'tomato_9']

Как можно заметить, это идеально подходит для случаев, когда данных в виде текстовых или изображений очень много и все они разбросаны по директориям. Тогда можно написать простой bash скрипт, уложив всё по папкам, и использовать эти функции.

## Предобработка данных

Как только ваши данные приобрели вид строковых/числовых Numpy массивов или `Dataset` объекта (или питоновского генератора), то это значит, что такой объект способен "выдавать" порции данных в виде строковых/числовых тензоров. Теперь настолько время **предобработки** данных:
* Токенизация строковых данных с последующей индексацией токенов
* Масштабирование признаков (нормализация)
* Приведение данных к значениям близким к нулю (этого требуют нейросети - обычно данные должны быть с 0-м мат.ожиданием и 1-чной дисперсией, либо в диапазоне [0, 1])

Мы должны стремиться к тому, чтобы препроцессинг данных осуществлялся **прямо в модели**, а не в виде какого-то внешнего пайплайна. Всё потому, что внешняя предобработка делает нашу модель менее переносимой, особенно, когда она должна использоваться прямо в продакшне.

Собственно, если ваша модель портируется на какое-то мобильное приложение, а вам надо предобработать данные на конкретном языке, то вы должны учесть, что язык может поменяться. В таком случае, любое несоответствие с исходным предположением о данных, подаваемых модели, полностью сломает нам всю систему.

Поэтому, куда проще просто экспортировать модель (end-to-end), которая уже включает в себя препроцессинг. **Идеальной** моделью можем считать такую модель, которая на вход получает максимально близкие к сырым данные: т.е. изображения могут быть RGB с интенсивностью в диапазоне [0, 255], а тексты - в виде строк формата `utf-8`. По сути, это очень облегчает жизнь пользователю модели - он понятия не имеет о том, как проходит препроцессинг внутри.

### Слои предобработки Keras

Такой препроцессинг можно осуществить с помощью **слоёв предобработки**:
* Векторизацию строк текстов через слой `TextVectorization`
* Нормализацию признаков через слой `Normalization`
* Также они позволяют выполнить масштабирование изображений, обрезание данных или даже процесс увеличения датасета изображений (image data augmentation)

Все эти слои позволяют сделать вашу модель **переносимой**.

Слой препроцессинга запускается вызовом `layer.adapt(data)` на выборке данных.

In [11]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [19]:
X_train = np.array(['Мама мыла раму очень качественно.', 
                    'Я могу только сказать, что вы сильно заблуждаетесь.',
                    'Я могу купить этот дом, но со временем передумали.',
                    'Он достиг небывалых высот в своём деле - это достойно!']) 

In [30]:
vectorizer = TextVectorization(output_mode='int')

In [31]:
vectorizer.adapt(X_train)

In [32]:
vectorizer(X_train)

<tf.Tensor: shape=(4, 9), dtype=int64, numpy=
array([[30, 17, 12, 14, 19,  0,  0,  0,  0],
       [ 3,  2,  7,  9,  6, 26, 10, 20,  0],
       [ 3,  2, 18,  4, 23, 15,  8, 27, 13],
       [29, 22, 16, 25, 28, 11, 24,  5, 21]])>

In [33]:
print(vectorizer.get_vocabulary())

['', '[UNK]', 'могу', 'Я', 'этот', 'это', 'что', 'только', 'со', 'сказать', 'сильно', 'своём', 'раму', 'передумали', 'очень', 'но', 'небывалых', 'мыла', 'купить', 'качественно', 'заблуждаетесь', 'достойно', 'достиг', 'дом', 'деле', 'высот', 'вы', 'временем', 'в', 'Он', 'Мама']


Когда мы вызываем метод `adapt`, он обрабатывает переданные данные и создаёт "словарь".